In [1]:
import sys
sys.path.append('../')
import numpy as np
import matplotlib.pyplot as plt
from VQE import VQE
from GroupingAlgorithm import *
from utils import get_backend_conectivity, RandomHamiltonian
# Importing standard Qiskit libraries
from qiskit import IBMQ, QuantumCircuit
from qiskit import Aer
from qiskit.circuit.library import EfficientSU2
from qiskit.utils.quantum_instance import QuantumInstance
from qiskit.algorithms import NumPyMinimumEigensolver
from qiskit.ignis.mitigation.measurement import CompleteMeasFitter
from qiskit.algorithms.optimizers import SPSA
from IPython.display import display, clear_output

In this notebook we test the VQE class. First we load our IBM account and define the backends. 

In [2]:
IBMQ.load_account()
provider      = IBMQ.get_provider(hub='ibm-q-csic', group='internal', project='iff-csic') 
backend_paris = provider.get_backend('ibmq_montreal')
WC_paris      = get_backend_conectivity(backend_paris)
backend_sim   = Aer.get_backend('aer_simulator') # Backend for simulation
NUM_SHOTS = 2**13  # Number of shots for each circuit

We create a Hamiltonian to perform the test.

In [ ]:
# qubit_op, init_state = LiH(initial_state=True)
# num_qubits = qubit_op.num_qubits

In [ ]:
num_qubits = 2
qubit_op = RandomHamiltonian( num_qubits, 6 )
init_state = QuantumCircuit(num_qubits)
print( qubit_op )

We calculate the minimum energy of the Hamiltonian by a classical method.

In [ ]:
result_exact = NumPyMinimumEigensolver().compute_minimum_eigenvalue(qubit_op)
result_exact.eigenvalue 

We will employ a Hardware efficient variation form.

In [ ]:
entangled_layer = []
for qbs in WC_paris :
    if qbs[0]<qbs[1] and qbs[1]<num_qubits :
        entangled_layer.append(qbs)

ansatz = init_state.compose( EfficientSU2(num_qubits,['ry','rz'], entanglement=entangled_layer, reps=1 ) )
ansatz.draw(output='mpl')

We define a callback function, to have the behavior of the algorithm by iterations.

In [ ]:
def callback( evals, params, energy, extra  ):  
    display("{}, {}".format(evals,energy))
    clear_output(wait=True)
    parameters.append(params)
    evaluations.append(evals)

parameters  = []
evaluations = []
energies    = [] 

We select SPSA with 100 iterations as classical optimizer.

In [ ]:
optimizer = SPSA( maxiter=100, last_avg=1 )

Initial parameters for the optimization

In [ ]:
num_var = ansatz.num_parameters
pars = [0.01] * num_var

In [ ]:
result_exact.eigenvalue 

VQE with entangled grouping

In [ ]:
solver = VQE( ansatz, optimizer, pars, grouping = 'Entangled' )
results = solver.compute_minimum_eigenvalue(qubit_op)
print( results.eigenvalue )

VQE with TPB grouping

In [ ]:
solver = VQE( ansatz, optimizer, pars, grouping = 'TPB' )
results = solver.compute_minimum_eigenvalue(qubit_op)
print( results.eigenvalue )

VQE with entangled grouping and the connectivity of ibmq_paris

In [ ]:
solver = VQE( ansatz, optimizer, pars, grouping = 'Entangled', conectivity = WC_paris )
results = solver.compute_minimum_eigenvalue(qubit_op)
print( results.eigenvalue )

Testing callback function

In [ ]:
parameters  = []
evaluations = []
solver = VQE( ansatz, optimizer, pars, grouping = 'Entangled', callback=callback )
results = solver.compute_minimum_eigenvalue(qubit_op)
print( results.eigenvalue )

Testing simulation with noise model

In [ ]:
# parameters  = []
# evaluations = []
# optimizer   = SPSA( maxiter=100, last_avg=1 )
# solver = VQE( ansatz, optimizer, pars, grouping = 'Entangled', callback=callback, quantum_instance=backend_noise )
# results = solver.compute_minimum_eigenvalue(qubit_op)
# print( results.eigenvalue )

In [ ]:
# plt.plot( evaluations )

Testing quantum instance

In [ ]:
qi = QuantumInstance( backend_sim, shots = NUM_SHOTS )
parameters  = []
evaluations = []
optimizer   = SPSA( maxiter=100, last_avg=1 )
solver = VQE( ansatz, optimizer, pars, grouping = 'Entangled', quantum_instance=qi )
results = solver.compute_minimum_eigenvalue(qubit_op)
print( results.eigenvalue )

In [ ]:
print(results)

In [ ]:
solver._energy_evaluation

Testing another function of the VQE class

In [ ]:
solver._ansatz

In [ ]:
solver.construct_expectation( pars, qubit_op )[0].draw()

In [ ]:
solver.get_optimal_circuit().draw(output='mpl')

In [ ]:
solver.get_optimal_cost()

In [ ]:
solver.get_optimal_vector()

In [ ]:
# solver.get_prob_vector_for_params()

In [ ]:
# solver.get_probabilities_for_counts()

In [ ]:
solver._initial_point

In [ ]:
solver.optimal_params

In [ ]:
solver._optimizer

In [ ]:
solver._quantum_instance

In [ ]:
solver.supports_aux_operators()

In [ ]:
print( solver.print_settings() )

In [ ]:
import qiskit.tools.jupyter
%qiskit_version_table